# 패키지 설치

In [61]:
!pip3 install jwt
!pip3 install PyJWT
!pip3 install pyupbit

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


# 참조
- 링크를 통하여 API Key 발급
> https://upbit.com/service_center/open_api_guide
- API Reference
> https://docs.upbit.com/reference
- 파이썬을 이용한 비트코인 자동매매 
> https://github.com/sharebook-kr/book-cryptocurrency
> https://wikidocs.net/book/1665
- 파이썬 비트코인 투자 자동화 강의 코드
> https://github.com/youtube-jocoding/pyupbit-autotrade

### 해당 어플리케이션은 아직 미완성이므로 참조용으로만 사용하시길 바랍니다. 모든 투자는 본인의 선택이며 투자 손실에 대한 책임은 본인에게 있습니다.

In [154]:
access_key = '<YOUR ACCESSKEY>' # <YOUR ACCESSKEY> 경고 : 외부 노출 주의!!!
secret_key = '<YOUR SECRETKEY>' # <YOUR SECRETKEY> 경고 : 외부 노출 주의!!!
server_url = 'https://api.upbit.com'

In [155]:
import time
import pyupbit
import datetime

access = access_key
secret = secret_key

In [129]:
# 로그인
upbit = pyupbit.Upbit(access, secret)
print("Login Success")

Login Success


In [130]:
# 변동성 돌파 전략으로 매수 목표가 조회
def get_target_price(ticker, k):
    df = pyupbit.get_ohlcv(ticker, interval="day", count=2)
    target_price = df.iloc[0]['close'] + (df.iloc[0]['high'] - df.iloc[0]['low']) * k
    return target_price

In [131]:
# 시작 시간 조회
def get_start_time(ticker):
    df = pyupbit.get_ohlcv(ticker, interval="day", count=1)
    start_time = df.index[0]
    return start_time

In [132]:
# 5분봉 차트 15일 이동 평균선 조회
def get_ma15(ticker):
    df = pyupbit.get_ohlcv(ticker, interval="minute5", count=15)
    ma15 = df['close'].rolling(15).mean().iloc[-1]
    return ma15

In [133]:
# 이전 5분봉 차트 15일 이동 평균선 조회(하락 판단)
def get_ma15_pre(ticker):
    pre = datetime.datetime.now() - datetime.timedelta(minutes=5)
    df = pyupbit.get_ohlcv(ticker, interval="minute5", count=15, to= pre)
    ma15 = df['close'].rolling(15).mean().iloc[-1]
    return ma15

In [141]:
# 5분봉 차트 60일 이동 평균선 조회
def get_ma60(ticker):
    df = pyupbit.get_ohlcv(ticker, interval="minute5", count=60)
    ma15 = df['close'].rolling(60).mean().iloc[-1]
    return ma15

In [142]:
# 투자내역 조회
def get_balance(ticker):
    balances = upbit.get_balances()
    for b in balances:
        if b['currency'] == ticker:
            if b['balance'] is not None:
                return float(b['balance'])
            else:
                return 0
    return 0

In [143]:
# 현재가 조회
def get_current_price(ticker):
    return pyupbit.get_orderbook(tickers=ticker)[0]["orderbook_units"][0]["ask_price"]

In [144]:
get_ma15("KRW-BTC")

55201200.0

In [145]:
get_ma60("KRW-BTC")

54454433.333333336

In [146]:
get_target_price("KRW-BTC", 0.5)

57366500.0

In [ ]:
# 자동매매 시작
while True:
    try:
        now = datetime.datetime.now()
        start_time = get_start_time("KRW-BTC")
        end_time = start_time + datetime.timedelta(days=1)
        ma15 = get_ma15("KRW-BTC")
        ma15_pre = get_ma15_pre("KRW-BTC")
        ma60 = get_ma60("KRW-BTC")
        current_price = get_current_price("KRW-BTC")
        krw = get_balance("KRW")
        print("-----------------------------------------------------------------------------------")
        print("현재 시간 : " , now)
            
        if ma15 / ma60 >= 1 and ma15 / ma60 <= 1.01 and ma15_pre < ma15:
            if krw > 5500:
                    print("15일 이동평균선이 60일 이동평균선을 넘어섰으므로 매수 합니다.")
                    upbit.buy_market_order("KRW-BTC", krw*0.9995)
            else:
                print("원화가 부족하여 매수를 할 수 없습니다.")
        
        else:
            btc = get_balance("BTC")
            if btc > 0.00008 and ma15_pre - ma15 < 0:
                print("15일 이동평균선에 하락이 있어 매도합니다.")
                #upbit.sell_market_order("KRW-BTC", btc*0.9995) # 테스트 중

        print("보유 BTC 수량" , btc, "/ 보유 원화",  krw, "원", "/ 골든크로스 상태 : ", ma15 / ma60)
        time.sleep(60)
    
    except Exception as e:
        print(e)
        time.sleep(60)

-----------------------------------------------------------------------------------
현재 시간 :  2021-05-18 10:55:33.760283
보유 BTC 수량 0 / 보유 원화 5997.10437954 원 / 골든크로스 상태 :  1.0198613449817717
-----------------------------------------------------------------------------------
현재 시간 :  2021-05-18 10:56:34.235529
보유 BTC 수량 0 / 보유 원화 5997.10437954 원 / 골든크로스 상태 :  1.0198170619980993
-----------------------------------------------------------------------------------
현재 시간 :  2021-05-18 10:57:34.728705
보유 BTC 수량 0 / 보유 원화 5997.10437954 원 / 골든크로스 상태 :  1.0198125114710523
-----------------------------------------------------------------------------------
현재 시간 :  2021-05-18 10:58:35.168087
보유 BTC 수량 0 / 보유 원화 5997.10437954 원 / 골든크로스 상태 :  1.0198079609301087
-----------------------------------------------------------------------------------
현재 시간 :  2021-05-18 10:59:35.683246
보유 BTC 수량 0 / 보유 원화 5997.10437954 원 / 골든크로스 상태 :  1.019810379820592
--------------------------------------------------------